In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd drive/MyDrive/jozef/DL4BIOMED

/content/drive/MyDrive/jozef/DL4BIOMED


In [ ]:
zip_file_path = 'jozef.zip'
extract_to_dir = 'project/'

In [ ]:
!unzip "$zip_file_path" -d "$extract_to_dir"

In [ ]:
%cd project/jozef/DL4BIOMED/

/content/project/jozef/DL4BIOMED


In [5]:
!python -m pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 116.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 127.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.8/529.8 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

Reboot kernel

for swissprot or tabula_muris edit main.yaml by changing the dataset field

For Tabula Muris

In [19]:
import hydra
from hydra import compose, initialize
import wandb
from hydra.utils import instantiate
from math import ceil
from omegaconf import OmegaConf
from prettytable import PrettyTable
import torch
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from sklearn.decomposition import PCA
# Get hydra config as omegaconf object

config_path = "./conf"
with initialize(config_path=config_path):
    cfg = compose("main.yaml")

<ipython-input-19-749289c7404f>:14: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=config_path):
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'main.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [16]:
train_dataset = instantiate(cfg.dataset.set_cls, mode='train')
samples,_ = train_dataset.load_tabular_muris()
samples = torch.from_numpy(samples)

/content/drive/MyDrive/jozef/DL4BIOMED/datasets/cell/utils.py:66: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['label'] = pd.Categorical(values=truth_labels)
/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_simple.py:373: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/content/drive/MyDrive/jozef/DL4BIOMED/datasets/cell/utils.py:66: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self.adata.obs['label'] = pd.Categorical(values=truth_labels)
/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_simple.py:373: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [ ]:

dist_matrix = torch.cdist(samples.T, samples.T, p=2)

# Convert distance matrix to NumPy for hierarchical clustering
dist_matrix_np = dist_matrix.numpy()

# Use scipy's linkage function to perform hierarchical clustering
# using the distance matrix
Z = hierarchy.linkage(dist_matrix_np, method='average')

# Plot dendrogram
plt.figure(figsize=(8, 4))
dn = hierarchy.dendrogram(Z)
plt.show()

In [ ]:
torch.save(Z, "./data/comet/tabula_muris/linkage.pt")

In [21]:
pca = PCA(whiten=True)
pca.fit(samples)
torch.save(pca.components_, "TM_PCA_components.pt")

In [ ]:
torch.save(pca.components_, "./data/comet/tabula_muris/TM_PCA_components.pt")

For swissprot

In [ ]:
import hydra
from hydra import compose, initialize
import wandb
from hydra.utils import instantiate
from math import ceil
from omegaconf import OmegaConf
from prettytable import PrettyTable
import torch
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from sklearn.decomposition import PCA
# Get hydra config as omegaconf object

config_path = "./conf"
with initialize(config_path=config_path):
    cfg = compose("main.yaml")

In [ ]:
train_dataset = instantiate(cfg.dataset.set_cls, mode='train')
samples= train_dataset.load_swissprot()

In [ ]:
data = []
for elem in samples:
  data.append(elem.input_seq.reshape(1,-1))
data = torch.cat(data,dim=0)

In [ ]:
pca = PCA(whiten=True)
pca.fit(data)
torch.save(pca.components_, "./data/comet/swissprot/SP_PCA_components.pt")

In [10]:
dist_matrix = torch.cdist(data.T, data.T, p=2)

# Convert distance matrix to NumPy for hierarchical clustering
dist_matrix_np = dist_matrix.numpy()

# Use scipy's linkage function to perform hierarchical clustering
# using the distance matrix
Z = hierarchy.linkage(dist_matrix_np, method='average')

# Plot dendrogram
plt.figure(figsize=(8, 4))
dn = hierarchy.dendrogram(Z)
plt.show()

NameError: ignored

In [ ]:
torch.save(torch.from_numpy(Z),'./data/comet/swissprot/linkage.pt')